In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Machine Translation is one of the most challenging tasks in Artificial Intelligence that works by investigating the use of software to translate a text or speech from one language to another. In this Project, I will take you through Machine Translation using Neural networks.

At the end of this Project, you will learn to develop a machine translation model using Neural networks and python. I will use the English language as an input and we will train our Machine Translation model to give the output in the French language. Now let’s start with importing all the libraries that we need for this task:

In [5]:
import collections
import os
import helper
import numpy as np
#import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy


# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [6]:
def load_data(path):
    
    input_file = os.path.join(path)
    with open(input_file, 'r') as f:
        data = f.read()
    return data.split('\n')

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [7]:
english_sentences = load_data('/content/drive/MyDrive/Data science/Machine Translation Dataset/small_vocab_en.csv')
french_sentences = load_data('/content/drive/MyDrive/Data science/Machine Translation Dataset/small_vocab_fr.csv')

In [8]:
english_sentences

['new jersey is sometimes quiet during autumn , and it is snowy in april .',
 'the united states is usually chilly during july , and it is usually freezing in november .',
 'california is usually quiet during march , and it is usually hot in june .',
 'the united states is sometimes mild during june , and it is cold in september .',
 'your least liked fruit is the grape , but my least liked is the apple .',
 'his favorite fruit is the orange , but my favorite is the grape .',
 'paris is relaxing during december , but it is usually chilly in july .',
 'new jersey is busy during spring , and it is never hot in march .',
 'our least liked fruit is the lemon , but my least liked is the grape .',
 'the united states is sometimes busy during january , and it is sometimes warm in november .',
 'the lime is her least liked fruit , but the banana is my least liked .',
 'he saw a old yellow truck .',
 'india is rainy during june , and it is sometimes warm in november .',
 'that cat was my most l

In [9]:
french_sentences

["new jersey est parfois calme pendant l' automne , et il est neigeux en avril .",
 'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .',
 'california est généralement calme en mars , et il est généralement chaud en juin .',
 'les états-unis est parfois légère en juin , et il fait froid en septembre .',
 'votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .',
 "son fruit préféré est l'orange , mais mon préféré est le raisin .",
 'paris est relaxant en décembre , mais il est généralement froid en juillet .',
 'new jersey est occupé au printemps , et il est jamais chaude en mars .',
 'notre fruit est moins aimé le citron , mais mon moins aimé est le raisin .',
 'les états-unis est parfois occupé en janvier , et il est parfois chaud en novembre .',
 'la chaux est son moins aimé des fruits , mais la banane est mon moins aimé.',
 'il a vu un vieux camion jaune .',
 'inde est pluvieux en juin , et il est parfois chaud en novembre .

### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [10]:
for i in range(2):
    print('small_vocab_en Line {}: {} '.format(i+1, english_sentences[i]))
    print('small_vocab_fr Line {}: {}'.format(i+1, french_sentences[i]))

small_vocab_en Line 1: new jersey is sometimes quiet during autumn , and it is snowy in april . 
small_vocab_fr Line 1: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2: the united states is usually chilly during july , and it is usually freezing in november . 
small_vocab_fr Line 2: les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [11]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

In [12]:
print('{}:English words'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{}:unique words'.format(len(english_words_counter)))
print('English most 10 common words :{}'.format(list(zip(*english_words_counter.most_common(10)))[0]))

print('{}:French words'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{}:unique words'.format(len(french_words_counter)))
print('French most 10 common words :{}'.format(list(zip(*french_words_counter.most_common(10)))[0]))


1823250:English words
227:unique words
English most 10 common words :('is', ',', '.', 'in', 'it', 'during', 'the', 'but', 'and', 'sometimes')
1961295:French words
355:unique words
French most 10 common words :('est', '.', ',', 'en', 'il', 'les', 'mais', 'et', 'la', 'parfois')


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [14]:
def tokenizer(x):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    t = tokenizer.texts_to_sequences(x)
    return (t, tokenizer)
#tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenizer(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate (zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1 ))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))    

    

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [20]:
def pad (x, length=None):
    padding = pad_sequences(x, padding='post',maxlen=length)
    return(padding)
#tests.test_pad(pad)    

# Pad Tokenized output
test = pad(text_tokenized)
for sample_i ,(token_sent, pad_token) in enumerate(zip(text_tokenized, test)):
    print('sequence {} in x'.format(sample_i+1))
    print('Input:{}'.format(token_sent))
    print('Output:{}'.format(pad_token))

sequence 1 in x
Input:[1, 2, 4, 5, 6, 7, 1, 8, 9]
Output:[1 2 4 5 6 7 1 8 9 0]
sequence 2 in x
Input:[10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Output:[10 11 12  2 13 14 15 16  3 17]
sequence 3 in x
Input:[18, 19, 3, 20, 21]
Output:[18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [21]:
def preprocess(x,y):
    
    preprocess_x, x_tk = tokenizer(x)
    preprocess_y, y_tk = tokenizer(y)
    
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentence, preproc_french_sentence, english_tokenizer, french_tokenizer = preprocess(english_sentences, french_sentences)

max_english_sequence_len = preproc_english_sentence.shape[1]
max_french_sequence_len = preproc_french_sentence.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data preprocessed')
print('Max English sentence length:',max_english_sequence_len)
print('Max French sentence length:',max_french_sequence_len)
print('English Vocabulary size:', english_vocab_size)
print('French Vocabulary size:', french_vocab_size)


Data preprocessed
Max English sentence length: 15
Max French sentence length: 21
English Vocabulary size: 199
French Vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [22]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    #{id, word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits,1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [23]:
from keras.layers import Input
from keras.models import Sequential

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    model = Sequential([GRU(100, input_shape=(21,1),return_sequences = True),TimeDistributed(Dense(345,activation='softmax'))])
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(0.001),metrics=['accuracy'])
    return model

    # Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentence, max_french_sequence_len) 
tmp_x = tmp_x.reshape(-1, preproc_french_sentence.shape[-2], 1)
#Train the neural network
simple_rnn_model = simple_model(tmp_x.shape, max_french_sequence_len,english_vocab_size,french_vocab_size)
                               
                               
simple_rnn_model.summary()

simple_rnn_model.fit(tmp_x, preproc_french_sentence, epochs=50, batch_size=1024, validation_split=0.2)
 


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_3 (GRU)                  (None, 21, 100)           30900     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 345)           34845     
Total params: 65,745
Trainable params: 65,745
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
108/108 [==============================] - 3s 20ms/step - loss: 4.0379 - accuracy: 0.3958 - val_loss: 2.4444 - val_accuracy: 0.4636
Epoch 2/50
108/108 [==============================] - 2s 16ms/step - loss: 2.3559 - accuracy: 0.4761 - val_loss: 2.0603 - val_accuracy: 0.5361
Epoch 3/50
108/108 [==============================] - 2s 16ms/step - loss: 1.9736 - accuracy: 0.5499 - val_loss: 1.7471 - val_accuracy: 0.5833
Epoch 4/50
108/108 [==============================] - 2s 16ms/step - loss: 1.7042 -

In [ ]:
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme en mois de il et il est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [ ]:
tmp_x[:1][0]

array([[17],
       [23],
       [ 1],
       [ 8],
       [67],
       [ 4],
       [39],
       [ 7],
       [ 3],
       [ 1],
       [55],
       [ 2],
       [44],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0]], dtype=int32)

### Model 2: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [28]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
#tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentence, preproc_french_sentence.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentence.shape[-2], 1))

bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentence.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)


bidi_model.fit(tmp_x, preproc_french_sentence, batch_size=1024, epochs=50, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/50
108/108 [==============================] - 10s 60ms/step - loss: 3.5275 - accuracy: 0.4509 - val_loss: 1.8506 - val_accuracy: 0.5579
Epoch 2/50
108/108 [==============================] - 6s 55ms/step - loss: 1.8102 - accuracy: 0.5647 - val_loss: 1.5179 - val_accuracy: 0.6025
Epoch 3/50
108/108 [==============================] - 6s 53ms/step - loss: 1.5248 - accuracy: 0.6034 - val_loss: 1.3963 - val_accuracy: 0.6146
Epoch 4/50
108/108 [==============================] - 6s 53ms/step - loss: 1.3951 - accuracy: 0.6179 - val_loss: 1.3354 - val_accuracy: 0.6153
Epoch 5/50
108/108 [==============================] - 6s 53ms/step - loss: 1.3194 - accuracy: 0.6310 - val_loss: 1.3013 - val_accuracy: 0.6188
Epoch 6/50
108/108 [==============================] - 6s 53ms/step - loss: 1.2649 - accuracy: 0.6417 - val_loss: 1.2886 - val_accuracy: 0.6188
Epoch 7/50
108/108 [==============================] - 6s 53ms/step - loss: 1.2245 - accuracy: 0.6490 - val_loss: 1.2816 - val_accuracy: 0.617

### Model 3: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [26]:
from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
#tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentence, max_french_sequence_len)
tmp_x = tmp_x.reshape((-1, preproc_french_sentence.shape[-2]))

# TODO: Train the neural network

embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_len,
    english_vocab_size,
    french_vocab_size)

embeded_model.fit(tmp_x, preproc_french_sentence, batch_size=1024, epochs=50, validation_split=0.2)


# TODO: Print prediction(s)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/50
108/108 [==============================] - 11s 40ms/step - loss: 4.6878 - accuracy: 0.3753 - val_loss: nan - val_accuracy: 0.4093
Epoch 2/50
108/108 [==============================] - 4s 34ms/step - loss: 2.8219 - accuracy: 0.4304 - val_loss: nan - val_accuracy: 0.5112
Epoch 3/50
108/108 [==============================] - 4s 35ms/step - loss: 2.1122 - accuracy: 0.5400 - val_loss: nan - val_accuracy: 0.6151
Epoch 4/50
108/108 [==============================] - 4s 34ms/step - loss: 1.5028 - accuracy: 0.6351 - val_loss: nan - val_accuracy: 0.7004
Epoch 5/50
108/108 [==============================] - 4s 36ms/step - loss: 1.1478 - accuracy: 0.7162 - val_loss: nan - val_accuracy: 0.7541
Epoch 6/50
108/108 [==============================] - 4s 34ms/step - loss: 0.9325 - accuracy: 0.7622 - val_loss: nan - val_accuracy: 0.7822
Epoch 7/50
108/108 [==============================] - 4s 34ms/step - loss: 0.7943 - accuracy: 0.7891 - val_loss: nan - val_accuracy: 0.8072
Epoch 8/50
108/108 

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [29]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
#tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentence)
tmp_x = tmp_x.reshape((-1, preproc_english_sentence.shape[1], 1))

encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentence.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encodeco_model.fit(tmp_x, preproc_french_sentence, batch_size=1024, epochs=50, validation_split=0.2)

print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/50
108/108 [==============================] - 9s 59ms/step - loss: 3.7225 - accuracy: 0.4054 - val_loss: 2.3681 - val_accuracy: 0.4936
Epoch 2/50
108/108 [==============================] - 6s 51ms/step - loss: 2.3126 - accuracy: 0.4998 - val_loss: 2.1673 - val_accuracy: 0.5060
Epoch 3/50
108/108 [==============================] - 6s 51ms/step - loss: 2.1049 - accuracy: 0.5155 - val_loss: 1.8477 - val_accuracy: 0.5538
Epoch 4/50
108/108 [==============================] - 5s 51ms/step - loss: 1.7942 - accuracy: 0.5589 - val_loss: 1.6787 - val_accuracy: 0.5682
Epoch 5/50
108/108 [==============================] - 6s 51ms/step - loss: 1.6559 - accuracy: 0.5714 - val_loss: 1.6049 - val_accuracy: 0.5759
Epoch 6/50
108/108 [==============================] - 5s 50ms/step - loss: 1.5826 - accuracy: 0.5820 - val_loss: 1.5263 - val_accuracy: 0.5930
Epoch 7/50
108/108 [==============================] - 5s 50ms/step - loss: 1.5137 - accuracy: 0.5932 - val_loss: 1.4714 - val_accuracy: 0.6032

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [31]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
#tests.test_model_final(model_final)

# OPTIONAL: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentence)
tmp_x = tmp_x.reshape((-1, preproc_english_sentence.shape[1], 1))

model_final = model_final(
    tmp_x.shape,
    preproc_french_sentence.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

model_final.fit(tmp_x, preproc_french_sentence, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(model_final.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')
# TODO: Train the final model

Epoch 1/10
108/108 [==============================] - 30s 223ms/step - loss: 2.8230 - accuracy: 0.4547 - val_loss: 1.2363 - val_accuracy: 0.6742
Epoch 2/10
108/108 [==============================] - 23s 211ms/step - loss: 1.0627 - accuracy: 0.7058 - val_loss: 0.6756 - val_accuracy: 0.7983
Epoch 3/10
108/108 [==============================] - 23s 212ms/step - loss: 0.5917 - accuracy: 0.8231 - val_loss: 0.4080 - val_accuracy: 0.8747
Epoch 4/10
108/108 [==============================] - 23s 212ms/step - loss: 0.3072 - accuracy: 0.9094 - val_loss: 0.2120 - val_accuracy: 0.9380
Epoch 5/10
108/108 [==============================] - 23s 212ms/step - loss: 0.1884 - accuracy: 0.9446 - val_loss: 0.1735 - val_accuracy: 0.9483
Epoch 6/10
108/108 [==============================] - 23s 211ms/step - loss: 0.1581 - accuracy: 0.9528 - val_loss: 0.1352 - val_accuracy: 0.9598
Epoch 7/10
108/108 [==============================] - 23s 211ms/step - loss: 0.1136 - accuracy: 0.9660 - val_loss: 0.1237 - val_ac

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?